In [20]:
%load_ext autoreload
%autoreload 2

import glob
import numpy as np
import pandas as pd
from snowflake.snowpark import Session
import json
import shutil
from sklearn.preprocessing import MinMaxScaler
from tqdm.autonotebook import tqdm
import seaborn as sns

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
nhits = pd.read_csv("tasty_csv_100.csv")
ens1 = pd.read_csv("eat_my_csv.csv")
ens1.columns = ens1.columns.str.lower()
nhits

,date,dc_name,size_code,total_tires
0,2022-09-20,BAKERSFIELD,1856015,7.869503
1,2022-09-21,BAKERSFIELD,1856015,8.427799
2,2022-09-22,BAKERSFIELD,1856015,11.679834
3,2022-09-23,BAKERSFIELD,1856015,5.988348
4,2022-09-24,BAKERSFIELD,1856015,10.681475
...,...,...,...,...
2725,2022-09-22,SAN JOSE,22570195,8.564729
2726,2022-09-23,SAN JOSE,22570195,7.607780
2727,2022-09-24,SAN JOSE,22570195,2.836328
2728,2022-09-25,SAN JOSE,22570195,1.485032


In [22]:
ens1

,date,dc_name,size_code,preds
0,2022-09-20,BAKERSFIELD,1856015,7.346239
1,2022-09-21,BAKERSFIELD,1856015,7.332078
2,2022-09-22,BAKERSFIELD,1856015,8.650714
3,2022-09-23,BAKERSFIELD,1856015,8.762070
4,2022-09-24,BAKERSFIELD,1856015,9.778122
...,...,...,...,...
2725,2022-09-22,SAN JOSE,22570195,8.927116
2726,2022-09-23,SAN JOSE,22570195,9.198637
2727,2022-09-24,SAN JOSE,22570195,3.920619
2728,2022-09-25,SAN JOSE,22570195,0.905336


In [23]:
merged = pd.merge(nhits, ens1, how="inner", on=["date", "dc_name", "size_code"])
merged

,date,dc_name,size_code,total_tires,preds
0,2022-09-20,BAKERSFIELD,1856015,7.869503,7.346239
1,2022-09-21,BAKERSFIELD,1856015,8.427799,7.332078
2,2022-09-22,BAKERSFIELD,1856015,11.679834,8.650714
3,2022-09-23,BAKERSFIELD,1856015,5.988348,8.762070
4,2022-09-24,BAKERSFIELD,1856015,10.681475,9.778122
...,...,...,...,...,...
2725,2022-09-22,SAN JOSE,22570195,8.564729,8.927116
2726,2022-09-23,SAN JOSE,22570195,7.607780,9.198637
2727,2022-09-24,SAN JOSE,22570195,2.836328,3.920619
2728,2022-09-25,SAN JOSE,22570195,1.485032,0.905336


In [24]:
merged["final_preds"] = merged[["total_tires", "preds"]].mean(axis=1)
merged

,date,dc_name,size_code,total_tires,preds,final_preds
0,2022-09-20,BAKERSFIELD,1856015,7.869503,7.346239,7.607871
1,2022-09-21,BAKERSFIELD,1856015,8.427799,7.332078,7.879938
2,2022-09-22,BAKERSFIELD,1856015,11.679834,8.650714,10.165274
3,2022-09-23,BAKERSFIELD,1856015,5.988348,8.762070,7.375209
4,2022-09-24,BAKERSFIELD,1856015,10.681475,9.778122,10.229798
...,...,...,...,...,...,...
2725,2022-09-22,SAN JOSE,22570195,8.564729,8.927116,8.745923
2726,2022-09-23,SAN JOSE,22570195,7.607780,9.198637,8.403209
2727,2022-09-24,SAN JOSE,22570195,2.836328,3.920619,3.378474
2728,2022-09-25,SAN JOSE,22570195,1.485032,0.905336,1.195184


In [27]:
merged.drop(columns=["total_tires", "preds"]).rename(columns={"final_preds": "total_tires"}).to_csv("after_lunch_csv.csv", index=False)

In [28]:
pd.read_csv("after_lunch_csv.csv")

,date,dc_name,size_code,total_tires
0,2022-09-20,BAKERSFIELD,1856015,7.607871
1,2022-09-21,BAKERSFIELD,1856015,7.879938
2,2022-09-22,BAKERSFIELD,1856015,10.165274
3,2022-09-23,BAKERSFIELD,1856015,7.375209
4,2022-09-24,BAKERSFIELD,1856015,10.229798
...,...,...,...,...
2725,2022-09-22,SAN JOSE,22570195,8.745923
2726,2022-09-23,SAN JOSE,22570195,8.403209
2727,2022-09-24,SAN JOSE,22570195,3.378474
2728,2022-09-25,SAN JOSE,22570195,1.195184
